In [1]:
'''
Worked on by: Meena Hari and Tarini Singh.

We perform data preprocessing using KNearestNeighbors.
66 new features are generated.

Trained a 1 layer ANN with transformed, higher dimensional 
dataset (each input consists of the raw board representaion 
(list of integers from 1 - 16) plus 66 newly generated features).

In prog.

'''

import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from tqdm import tqdm
import keras.backend as K
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Conv2D, Flatten, Input
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from keras.models import load_model
import keras.losses

from constants import * 
from heuristic import *
from io_help import *
from solver import *

def load_data(file_name):
	"""
	This function reads in training data from a file and returns 
	the boards in X and their labels in Y as a tuple. 
	"""
	file = open(file_name, "r")

	X = []
	Y = []

	for string in file: 
		(board, dist) = string_to_board_and_dist(string)
		X.append(np.asarray(board).flatten())
		Y.append(dist)
    

	file.close()

	X_train = np.asarray(X)
	Y_train = np.asarray(Y)
    
	return(X_train, Y_train)

Using TensorFlow backend.


In [3]:
# Load dataset. 
# X: board inputs, Y: true output.
(X,Y) = load_data('meena_5_18_2020_31931.txt')

In [4]:
model = NearestNeighbors(n_neighbors=50, n_jobs = -1).fit(X,Y)
pred = model.kneighbors(X)

In [6]:
# Generating features based on 50 nearest neighbors.

data_arr = np.zeros([len(X), 16*2*2 + 2])

for i in tqdm(range(len(X))):
    row = X[i]
    # Grabs the rows in X corresponding to 50 nearest neighbors of X[i].
    # pred[1][i] contains a list of the indices of the 50 nearest neighbors.
    data = X[pred[1][i]]
    # Divide X[i] by each of its neighbors. div should be a 
    # 50 x 16 matrix, i.e. div[j] = X[i] / X[j].
    div = (row / data)
    # Subtract X[i] by each of its neighbors. diff should be a 
    # 50 x 16 dimension matrix.
    diff = (row - data)
    # concat is a 50 x 32 matrix.
    concat = np.concatenate([div, diff], axis = 1)
    # means is a 50 x 32 matrix.
    # std is a 50 x 32 matrix.
    means, stds = np.nanmean(concat, axis = 0), np.nanstd(concat, axis = 0)
    # Populate data_arr with newly generated features.
    data_arr[i, :len(means)] = means
    data_arr[i, len(means):len(means) + len(stds)] = stds
    data_arr[i, -1] = np.nanmean(pred[0][i])
    data_arr[i, -2] = np.nanstd(pred[0][i])
    

100%|██████████| 78069/78069 [00:19<00:00, 4041.32it/s]


In [5]:
data_arr
# Following output makes sense because the first couple of rows represent boards 
# close to the solution (based on the way we ordered our training sets... first data points in X
# are 1, 2, and 3 moves away from solution), so div will be 1 for most entries, diff will be 0 for most
# entries, and so on. The last couple rows will represent boards farther from the solution, so the generated 
# features are not clean numbers..

array([[1.        , 1.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       [1.        , 1.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       [1.        , 1.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [1.02666667, 1.00571429, 0.987     , ..., 1.77189164, 1.36790934,
        8.05784239],
       [1.012     , 1.03333333, 0.995     , ..., 1.92675894, 1.34688168,
        8.16736859],
       [1.        , 0.99333333, 0.995     , ..., 2.2       , 1.26737256,
        7.27418496]])

In [6]:
# Concatenate generated features to the original dataset.
X2 = np.concatenate([X, data_arr], axis=1)

In [8]:
X_train = X2[5000:, :]
Y_train = Y[5000:]

'''
Yes, realized that we generated nearest neighbors on test 
points, which definitely influences training samples and
thus accuracy on test points. Tarini and I (Meena) will 
fix this by testing on a fresh dataset later in the week. 
We will also do some more hyperparamter tuning later.
Will keep this file updated. Thanks.
'''

X_test = X2[0:5000, :]
Y_test = Y[0:5000]

# Build Model
model = Sequential()

# Input Layer
model.add(Dense(units=(16*2*2+2+16), input_dim=(16*2*2+2+16), activation='relu'))
model.add(Dropout(0.1))

# Hidden Layers
model.add(Dense(units=66+16, activation='relu'))

# Output Layer
model.add(Dense(units=1, activation='linear'))

# Define the optimizer and loss function
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

# You can also define a custom loss function
# model.compile(optimizer='adam', loss=custom_loss)

# Train 
model.fit(X_train, Y_train, epochs=20)

# Test
score = model.evaluate(X_test, Y_test)

print(score)

Epoch 1/20
73069/73069 [==============================] - 3s 40us/step - loss: 12.2909 - accuracy: 0.1466
Epoch 2/20
73069/73069 [==============================] - 3s 37us/step - loss: 9.5735 - accuracy: 0.1819
Epoch 3/20
73069/73069 [==============================] - 3s 36us/step - loss: 8.8065 - accuracy: 0.2071
Epoch 4/20
73069/73069 [==============================] - 3s 37us/step - loss: 8.4595 - accuracy: 0.2288 0s
Epoch 5/20
73069/73069 [==============================] - 3s 37us/step - loss: 8.1653 - accuracy: 0.2415
Epoch 6/20
73069/73069 [==============================] - 3s 38us/step - loss: 7.8654 - accuracy: 0.2530
Epoch 7/20
73069/73069 [==============================] - 3s 38us/step - loss: 7.6670 - accuracy: 0.2588
Epoch 8/20
73069/73069 [==============================] - 3s 39us/step - loss: 7.5412 - accuracy: 0.2690 0s - l
Epoch 9/20
73069/73069 [==============================] - 3s 38us/step - loss: 7.3395 - accuracy: 0.2764
Epoch 10/20
73069/73069 [===================

In [10]:
Y.shape

(78069,)

In [25]:
for i in range (100):
        print("NN Heuristic: ", model.predict(X_test[i:(i+1),:]))
       # print("Manhattan Heuristic: ", manhattan(X[i], model))
        #(_, _, path) = solve(X[i], manhattan, None)
        print("True: ", Y[i])
        #print("True, Shortest Distance: ", len(path))
        print("\n")

NN Heuristic:  [[1.5596578]]
True:  1


NN Heuristic:  [[2.228024]]
True:  2


NN Heuristic:  [[3.4648104]]
True:  3


NN Heuristic:  [[3.8521435]]
True:  4


NN Heuristic:  [[5.4468412]]
True:  5


NN Heuristic:  [[6.736401]]
True:  6


NN Heuristic:  [[6.995502]]
True:  7


NN Heuristic:  [[7.353508]]
True:  8


NN Heuristic:  [[8.947558]]
True:  9


NN Heuristic:  [[1.973891]]
True:  1


NN Heuristic:  [[2.6151137]]
True:  2


NN Heuristic:  [[3.8712966]]
True:  3


NN Heuristic:  [[4.046033]]
True:  4


NN Heuristic:  [[5.4307375]]
True:  5


NN Heuristic:  [[6.0985365]]
True:  6


NN Heuristic:  [[7.152503]]
True:  7


NN Heuristic:  [[7.807934]]
True:  8


NN Heuristic:  [[8.438952]]
True:  9


NN Heuristic:  [[9.169146]]
True:  10


NN Heuristic:  [[11.997081]]
True:  11


NN Heuristic:  [[11.811531]]
True:  12


NN Heuristic:  [[13.422956]]
True:  13


NN Heuristic:  [[15.145297]]
True:  14


NN Heuristic:  [[16.832247]]
True:  15


NN Heuristic:  [[18.564203]]
True:  16


NN H

In [16]:
model.predict(X_test)


array([[ 1.5596578],
       [ 2.2280233],
       [ 3.4648097],
       ...,
       [25.047308 ],
       [ 1.5596573],
       [ 2.2280228]], dtype=float32)

In [21]:
X_test.shape
model.predict(X_test[1:2,:])

array([[2.228024]], dtype=float32)